Technical, trade-related, operational leasing & other business services analysed by continents and countries 2012 - 2016

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.ons.gov.uk/businessindustryandtrade/internationaltrade/datasets/internationaltradeinservicesreferencetables')
    tabs = { tab.name: tab for tab in scraper.distribution().as_databaker() }

In [2]:
tab = tabs['Table B1']

In [3]:
observations = tab.excel_ref('E7').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
Year = tab.excel_ref('E4').expand(RIGHT).is_not_whitespace()

In [5]:
Flow = tab.excel_ref('E3').expand(RIGHT).is_not_blank()

In [6]:
geo1 = tab.excel_ref('C7').expand(DOWN) - tab.excel_ref('C100').expand(DOWN)

In [7]:
geo2 = tab.excel_ref('B35').expand(DOWN) - tab.excel_ref('B100').expand(DOWN)  

In [8]:
geo3 = tab.excel_ref('A51').expand(DOWN) - tab.excel_ref('A100').expand(DOWN) 

In [9]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,ABOVE),
            HDim(geo1,'geo1',DIRECTLY,LEFT),
            HDim(geo2,'geo2',DIRECTLY,LEFT),
            HDim(geo3,'geo3',DIRECTLY,LEFT),
            HDim(Flow, 'Flow',CLOSEST,LEFT),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit','gbp-million')
            ]

In [10]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [11]:
new_table = c1.topandas()

In [12]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [13]:
new_table['Year'] = new_table['Year'].astype(int)

In [14]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [15]:
new_table['ONS Partner Geography'] = new_table['geo1'].fillna('') + new_table['geo2'].fillna('') + new_table['geo3'].fillna('')

In [16]:
new_table = new_table[new_table['Value'] != '' ]

In [17]:
new_table['Flow'] = new_table['Flow'].str.lower()

In [18]:
new_table['BOP Service'] = 'Technical, trade-related, operational leasing and other business services'

In [19]:
new_table['International Trade Basis'] = 'BOP'

In [20]:
new_table = new_table[['ONS Partner Geography', 'Year','Flow','BOP Service', 'International Trade Basis', 'Measure Type','Value','Unit' ]]

In [21]:
new_table['ONS Partner Geography'] = new_table['ONS Partner Geography'].map(
    lambda x: {
        'Total European Union (EU)' : 'Total EU (27)', 
        'Total EFTA' : 'EFTA',
        'Total Europe': 'Europe' ,
        'Total European Union (EU)': 'Total EU(28)',
        'TOTAL INTERNATIONAL TRADE IN SERVICES': 'Whole world'
        }.get(x, x))

In [22]:
# if is_interactive():
#     SubstancetinationFolder = Path('out')
#     SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
#     new_table.to_csv(SubstancetinationFolder / ('tableB1.csv'), index = False)

In [23]:
new_table.head()

,ONS Partner Geography,Year,Flow,BOP Service,International Trade Basis,Measure Type,Value,Unit
0,Austria,2012,exports,"Technical, trade-related, operational leasing ...",BOP,GBP Total,21.743,gbp-million
1,Austria,2013,exports,"Technical, trade-related, operational leasing ...",BOP,GBP Total,47,gbp-million
2,Austria,2014,exports,"Technical, trade-related, operational leasing ...",BOP,GBP Total,32.9446,gbp-million
3,Austria,2015,exports,"Technical, trade-related, operational leasing ...",BOP,GBP Total,24.3838,gbp-million
4,Austria,2016,exports,"Technical, trade-related, operational leasing ...",BOP,GBP Total,40,gbp-million
